In [14]:
#Importing data set using Pandas
import pandas as pd
csv_path = r"C:\Users\sansk\Downloads\Bank_Personal_Loan_Modelling(1).csv"

df = pd.read_csv(csv_path)
print(df.head(5))

   ID  Age  Experience  Income  ZIP Code  Family CCAvg  Education  Mortgage  \
0   1   25           1      49     91107       4  1/60          1         0   
1   2   45          19      34     90089       3  1/50          1         0   
2   3   39          15      11     94720       1  1/00          1         0   
3   4   35           9     100     94112       1  2/70          2         0   
4   5   35           8      45     91330       4  1/00          2         0   

   Personal Loan  Securities Account  CD Account  Online  CreditCard  
0              0                   1           0       0           0  
1              0                   1           0       0           0  
2              0                   0           0       0           0  
3              0                   0           0       0           0  
4              0                   0           0       0           1  


In [15]:
df.head(5)


,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1/60,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1/50,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1/00,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2/70,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1/00,2,0,0,0,0,0,1


In [16]:
# We can see that the dimension of the dataframe
# We have 5000 sample data from a Bank. Containing 14 column (Variables)
df.shape


(5000, 14)

In [17]:
'''About this file
ID: customer Id (continuous variable)

Age:customer's age in completed years (Categorical Variable)

Experience:years of professional experience (Catagorical)

Income:Annual income of the customer in thousands (Continious)

ZIP Code:Address Zip code (Continuous)

Family:family size of customer (Catagorical)

CCAvg:Avg. spending on credit card per month
due to all other parameter is annually so i converted it for per year

Education :Educational level ( undergraduate= 1 , graduate=2 , advance=3)

Education :Educational level ( undergraduate= 1 , graduate=2 , advance=3)

Personal Loan: Did this customer accept the personal loan offered in the last campaign?

Securities Account:Does the customer have a securities account with the bank?

CD Account:Does the customer have a certificate of deposit(CD) account with the bank?

Online:Does the customer use internet banking facilities

CreditCard: Does the customer use a credit card issued by universalBank?'''

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  5000 non-null   int64 
 1   Age                 5000 non-null   int64 
 2   Experience          5000 non-null   int64 
 3   Income              5000 non-null   int64 
 4   ZIP Code            5000 non-null   int64 
 5   Family              5000 non-null   int64 
 6   CCAvg               5000 non-null   object
 7   Education           5000 non-null   int64 
 8   Mortgage            5000 non-null   int64 
 9   Personal Loan       5000 non-null   int64 
 10  Securities Account  5000 non-null   int64 
 11  CD Account          5000 non-null   int64 
 12  Online              5000 non-null   int64 
 13  CreditCard          5000 non-null   int64 
dtypes: int64(13), object(1)
memory usage: 547.0+ KB


In [18]:
#Since the data column "CCAvg" is in the form of object, we can see that this is a catagorical variable
df['CCAvg'].describe()

count     5000
unique     108
top       0/30
freq       241
Name: CCAvg, dtype: object

In [19]:
#Check the statistical dimentions of the data set
df.describe()

,ID,Age,Experience,Income,ZIP Code,Family,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000
mean,2500.500000,45.338400,20.104600,73.774200,93152.503000,2.396400,1.881000,56.498800,0.096000,0.104400,0.06040,0.596800,0.294000
std,1443.520003,11.463166,11.467954,46.033729,2121.852197,1.147663,0.839869,101.713802,0.294621,0.305809,0.23825,0.490589,0.455637
min,1.000000,23.000000,-3.000000,8.000000,9307.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1250.750000,35.000000,10.000000,39.000000,91911.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,2500.500000,45.000000,20.000000,64.000000,93437.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.00000,1.000000,0.000000
75%,3750.250000,55.000000,30.000000,98.000000,94608.000000,3.000000,3.000000,101.000000,0.000000,0.000000,0.00000,1.000000,1.000000
max,5000.000000,67.000000,43.000000,224.000000,96651.000000,4.000000,3.000000,635.000000,1.000000,1.000000,1.00000,1.000000,1.000000


In [20]:
miss_data = df.isnull()

In [21]:
miss_data.head(5)

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [22]:
# Checking for any missing values
for column in miss_data.columns.values.tolist():
    print(column)
    print(miss_data[column].value_counts())
    print(" ")

ID
ID
False    5000
Name: count, dtype: int64
 
Age
Age
False    5000
Name: count, dtype: int64
 
Experience
Experience
False    5000
Name: count, dtype: int64
 
Income
Income
False    5000
Name: count, dtype: int64
 
ZIP Code
ZIP Code
False    5000
Name: count, dtype: int64
 
Family
Family
False    5000
Name: count, dtype: int64
 
CCAvg
CCAvg
False    5000
Name: count, dtype: int64
 
Education
Education
False    5000
Name: count, dtype: int64
 
Mortgage
Mortgage
False    5000
Name: count, dtype: int64
 
Personal Loan
Personal Loan
False    5000
Name: count, dtype: int64
 
Securities Account
Securities Account
False    5000
Name: count, dtype: int64
 
CD Account
CD Account
False    5000
Name: count, dtype: int64
 
Online
Online
False    5000
Name: count, dtype: int64
 
CreditCard
CreditCard
False    5000
Name: count, dtype: int64
 


SyntaxError: invalid syntax (3528599804.py, line 1)